In [0]:
%run ./project_config

In [0]:
%run ./parameters

In [0]:
from pyspark.sql import functions as f, DataFrame
from pyspark.sql.window import Window
from functions import load_table, save_table, read_csv_file, apply_inclusion_criteria, map_column_values
from functions.functions import union_dataframe_list


# 1 Load tables

In [0]:
cohort_filtered = load_table('cohort_filtered')
display(cohort_filtered.limit(100))

gdppr = load_table('gdppr', method='gdppr')
display(gdppr.limit(100))

# 2 Codelists

In [0]:
dict_codelists_snomed = {
    "bmi": "./codelists/bmi_snomed.csv",
    "height": "./codelists/height_snomed.csv",
    "weight": "./codelists/weight_snomed.csv",
}

dict_min_values = {
    "bmi": 10,
    "height": 50,
    "weight": 25
}

dict_max_values = {
    "bmi": 270,
    "height": 275,
    "weight": 650
}

list_codelists_snomed = [
    read_csv_file(codelist_path)
    .withColumn('phenotype', f.lit(phenotype))
    for phenotype, codelist_path in dict_codelists_snomed.items()
]

codelist_snomed = spark.createDataFrame(union_dataframe_list(list_codelists_snomed))

codelist_snomed = (
    codelist_snomed
    .transform(
        map_column_values,
        map_dict=dict_min_values, column='phenotype', new_column='min_value'
    )
    .transform(
        map_column_values,
        map_dict=dict_max_values, column='phenotype', new_column='max_value'
    )
)

display(codelist_snomed)


# 3 Prepare datasets

In [0]:
gdppr_prepared = (
    gdppr
    .select(
        'person_id', 'date', 'code',
        f.col('value1_condition').alias('value')
    )
)

# 4 Cohort dates

In [0]:
cohort_prepared = (
    cohort_filtered
    .select(
        'person_id',
        f.date_sub('index_mi_date', 2*365).alias('min_date'),
        f.date_add('index_mi_date', 3*30).alias('max_date'),
        f.col('index_mi_date').alias('target_date')
    )
)

# 5 Perform matching

In [0]:
measures_code_matched = (
    gdppr_prepared
    .join(
        f.broadcast(codelist_snomed),
        on='code', how='inner'
    )
    .filter(f.col('value').between(f.col('min_value'), f.col('max_value')))
)

measures_cohort_matched = (
    measures_code_matched
    .join(
        cohort_prepared,
        on='person_id', how='inner'
    )
    .filter(f.col('date').between(f.col('min_date'), f.col('max_date')))
)


# 6 Aggregate and pivot wide

In [0]:
_window = Window.partitionBy('person_id', 'phenotype').orderBy(f.col('abs_date_diff'))

measures_aggregated = (
    measures_cohort_matched
    .withColumn('abs_date_diff', f.abs(f.datediff('date', 'target_date')))
    .withColumn('rank', f.rank().over(_window))
    .filter(f.col('rank') == 1)
)

measures_wide = (
    measures_aggregated
    .groupBy('person_id')
    .pivot('phenotype')
    .agg(
        f.first('value').alias('value'),
        f.first('date').alias('date')
    )
)

# 7 Save table

In [0]:
cohort_measures = (
    cohort_filtered
    .select('person_id')
    .join(measures_wide, on='person_id', how='left')
)

save_table(cohort_measures, 'cohort_measures')

# 8 Display

In [0]:
cohort_measures = load_table('cohort_measures')
display(cohort_measures.limit(100))